<img src= "https://maua.br/images/logo-IMT.png" width=250>

#  <center> <font color=#023e8a>  **ECM797: Introdução à Computação Quântica** <center>
 <center> Prof. Sandro Martini <center>

## <center>  Semana 17: Qiskit - **Multiplicações usando a Transformada de Fourier Quântica**

## Nome: Rafael Rubio

## Objetivos

- Entender como a Transformada de Fourier Quântica pode ser usada para multiplicar dois números.

# Introdução

Neste notebook, vamos explorar a implementação de multiplicação quântica com o Qiskit. Em computadores clássicos, a multiplicação é feita por meio de circuitos digitais chamados multiplicadores binários. Em computadores quânticos, utilizamos diferentes abordagens, como a Transformada de Fourier Quântica (*Quantum Fourier Transfor*, QFT).

Para entender a QFT, é útil compará-la com a Transformada Rápida de Fourier (FFT) clássica. A FFT é um algoritmo eficiente para computar a Transformada de Fourier Discreta (DFT), que transforma uma sequência de números em uma soma de funções sinusoidais. A FFT é amplamente utilizada em processamento de sinais, análise de imagem e resolução de equações diferenciais.

A QFT é a versão quântica da FFT e desempenha um papel crucial em muitos algoritmos quânticos, incluindo a fatoração de números inteiros e a multiplicação de números. A QFT é constituída por uma série de operações que transformam o estado quântico de entrada em um estado quântico que representa a transformada de Fourier dos coeficientes de entrada. 

## Transformada de Fourier Quântica (QFT)

A Transformada de Fourier Quântica (QFT) em 𝑛 qubits é a operação:

$$
|j\rangle \mapsto \frac{1}{2^{n / 2}} \sum_{k=0}^{2^n-1} e^{2 \pi i j k / 2^n}|k\rangle
$$

O circuito que implementa essa transformação pode ser realizado usando portas de Hadamard em cada qubit, uma série de portas controladas-U1 (ou Z, dependendo da fase) e uma camada de portas Swap. A camada de portas Swap pode, em princípio, ser dispensada se a QFT aparecer no final do circuito, pois então a reordenação pode ser feita classicamente. 

### Diagrama de Circuito QFT:


![FIG](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*LKcMiuZgHrApIa9M2qVePg.png)

### Explicação do Circuito:

1. **Qubits de Entrada**: O circuito QFT recebe qubits em um estado de superposição ou um estado específico de entrada.

2. **Portas Quânticas**: A QFT consiste em uma série de portas Hadamard (H) e portas de fase controladas (CRk), que transformam o estado dos qubits.

3. **Qubits de Saída**: O estado de saída do circuito QFT é uma superposição dos estados de entrada, com coeficientes de fase ajustados conforme a transformada de Fourier.

O que entra no circuito são os qubits representando um estado de entrada, por exemplo,  ∣x⟩. O que sai é a transformada de Fourier desse estado, ∣x~⟩, que codifica as frequências de Fourier dos coeficientes de entrada.

### Como a QFT realiza a multiplicação de dois números

1. **Codificação dos Operandos**:

Os dois números a serem multiplicados são codificados em qubits. Por exemplo, se quisermos multiplicar 2 e 3, representamos 2 como 10 em binário e 3 como 11 em binário.

2. **Aplicação da QFT nos Operandos**:

A QFT é aplicada aos qubits que representam os operandos. Esta etapa transforma os números binários em seus correspondentes no domínio da frequência quântica.

3. **Realização de Somas no Domínio da Frequência**:

No domínio da frequência, as operações de adição são realizadas. A QFT transforma a multiplicação no domínio do tempo em uma série de somas no domínio da frequência, que é uma operação mais simples de ser realizada quânticamente.

4. **Aplicação da QFT Inversa (QFT†)**:

Após a realização das somas, aplicamos a QFT inversa para transformar os resultados de volta para o domínio do tempo. Esta etapa converte as somas realizadas no domínio da frequência de volta em uma multiplicação no domínio do tempo.

5. **Medida dos Qubits de Saída**:

Os qubits que armazenam o resultado final são medidos, e o resultado da multiplicação é extraído dos bits clássicos.

### Exemplo Prático: Multiplicação de 2 por 3

Vamos implementar um exemplo prático onde multiplicamos 2 por 3 usando Qiskit.

**Passo 1**: Inicializar os Registradores e o Circuito Quântico

Vamos configurar os registradores quânticos e clássicos. Usaremos 8 qubits e 4 bits clássicos:

In [25]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import RGQFTMultiplier
from qiskit import transpile
from qiskit_aer import Aer

# Definindo os registros quânticos e clássicos
q = QuantumRegister(8, 'q')
c = ClassicalRegister(4, 'c')
circuit = QuantumCircuit(q, c)

**Passo 2**: Codificar os Valores nos Operandos

Codificamos os valores dos operandos nos qubits apropriados:

In [26]:
# Definindo os operandos A = 10 (binário '10') e B = 11 (binário '11')
circuit.x(q[1])  # A
circuit.x(q[2])  # B
circuit.x(q[3])  # B

**Explicação**:

O operador `circuit.x(q[n])` aplica a porta X (NOT) ao qubit n, mudando seu estado de 0 para 1.

**Passo 3**: Criar o Circuito de Multiplicação QFT

Usamos o circuito RGQFTMultiplier para realizar a multiplicação

In [27]:
# Adicionando o multiplicador RGQFT ao circuito
circuit1 = RGQFTMultiplier(num_state_qubits=2, num_result_qubits=4)
circuit = circuit.compose(circuit1)

**Explicação**:

O RGQFTMultiplier é uma implementação de um multiplicador quântico que utiliza a QFT.

**Passo 4**: Medir os Qubits de Saída

Medimos os qubits que armazenam o resultado da multiplicação:

In [28]:
# Medindo os qubits de resultado
circuit.measure(q[4], c[0])
circuit.measure(q[5], c[1])
circuit.measure(q[6], c[2])
circuit.measure(q[7], c[3])

display(circuit.draw(output='text'))

┌──────────────────┐            
q_0: ─────┤0                 ├────────────
     ┌───┐│                  │            
q_1: ┤ X ├┤1                 ├────────────
     ├───┤│                  │            
q_2: ┤ X ├┤2                 ├────────────
     ├───┤│                  │            
q_3: ┤ X ├┤3                 ├────────────
     └───┘│  RGQFTMultiplier │┌─┐         
q_4: ─────┤4                 ├┤M├─────────
          │                  │└╥┘┌─┐      
q_5: ─────┤5                 ├─╫─┤M├──────
          │                  │ ║ └╥┘┌─┐   
q_6: ─────┤6                 ├─╫──╫─┤M├───
          │                  │ ║  ║ └╥┘┌─┐
q_7: ─────┤7                 ├─╫──╫──╫─┤M├
          └──────────────────┘ ║  ║  ║ └╥┘
c: 4/══════════════════════════╩══╩══╩══╩═
                               0  1  2  3

**Explicação**:

A operação `circuit.measure(q[n], c[m])` mede o qubit n e armazena o resultado no bit clássico m.

**Passo 5**: Executar o Circuito e Obter os Resultados

Enviamos o circuito para um simulador quântico e obtemos os resultados:

In [29]:
# Escolhendo o backend
backend = Aer.get_backend('qasm_simulator')

# Otimizando o circuito para o backend escolhido
transpiled_circuit = transpile(circuit, backend)

# Executando o circuito otimizado no backend
job = backend.run(transpiled_circuit, shots=2000)

# Obtendo o resultado do trabalho executado
result = job.result()

# Obtendo as contagens das medições
counts = result.get_counts()

print('2*3')
print('----')
print(counts)

2*3
----
{'0110': 2000}


**Explicação**:

Utilizamos o simulador qasm_simulator para executar o circuito e obtemos as contagens das medições. Observe que o número binário `0110`, corresponde ao número `6`em decimal.

<div class="alert alert-success"> <strong> &#x1F4DD; Atividade 1:</strong>  Altere os valores dos operandos para outros números e observe como o circuito se comporta. Por exemplo, tente multiplicar 3×4 ou 5×6.

In [30]:
q2 = QuantumRegister(12, 'q')
c2 = ClassicalRegister(6, 'c')
circuit2 = QuantumCircuit(q2, c2) 

# A (q[0] LSB, q[1], q[2] MSB)
circuit2.x(q2[0])
circuit2.x(q2[2])

# B (q[3] LSB, q[4], q[5] MSB)
circuit2.x(q2[4])
circuit2.x(q2[5]) 

mult = RGQFTMultiplier(num_state_qubits=3, num_result_qubits=6)
circuit2 = circuit2.compose(mult) 

for i in range(6):
    circuit2.measure(q2[6 + i], c2[i])

# Visualizar o circuito
display(circuit2.draw(output='text'))

┌───┐┌───────────────────┐                  
 q_0: ┤ X ├┤0                  ├──────────────────
      └───┘│                   │                  
 q_1: ─────┤1                  ├──────────────────
      ┌───┐│                   │                  
 q_2: ┤ X ├┤2                  ├──────────────────
      └───┘│                   │                  
 q_3: ─────┤3                  ├──────────────────
      ┌───┐│                   │                  
 q_4: ┤ X ├┤4                  ├──────────────────
      ├───┤│                   │                  
 q_5: ┤ X ├┤5                  ├──────────────────
      └───┘│   RGQFTMultiplier │┌─┐               
 q_6: ─────┤6                  ├┤M├───────────────
           │                   │└╥┘┌─┐            
 q_7: ─────┤7                  ├─╫─┤M├────────────
           │                   │ ║ └╥┘┌─┐         
 q_8: ─────┤8                  ├─╫──╫─┤M├─────────
           │                   │ ║  ║ └╥┘┌─┐      
 q_9: ─────┤9                  ├─╫──╫──╫─┤M├──────
           │                   │ ║  ║  ║ └╥┘┌─┐   
q_10: ─────┤10                 ├─╫──╫──╫──╫─┤M├───
           │                   │ ║  ║  ║  ║ └╥┘┌─┐
q_11: ─────┤11                 ├─╫──╫──╫──╫──╫─┤M├
           └───────────────────┘ ║  ║  ║  ║  ║ └╥┘
 c: 6/═══════════════════════════╩══╩══╩══╩══╩══╩═
                                 0  1  2  3  4  5

In [31]:
# Passo 5: Executar o Circuito e Obter os Resultados
backend2 = Aer.get_backend('qasm_simulator')
transpiled2 = transpile(circuit2, backend2)
job2 = backend2.run(transpiled2, shots=2000)
result2 = job2.result()
counts2 = result2.get_counts()

print('5 * 6 =')
print('-----')
print(counts2)

5 * 6 =
-----
{'011110': 2000}


<div class="alert alert-success"> <strong> &#x1F4DD; Atividade 2:</strong>  Descreva e explique como seria realizada a multiplicação de dois números usando portas lógicas clássicas.

**Resposta** 
A multiplicação de dois números usando portas lógicas clássicas é realizada por meio de multiplicadores binários. São utilizadas portas lógicas AND, OR e NOT para implementar a lógica da multiplicação. O processo envolve a decomposição dos números em seus bits binários e a aplicação de operações lógicas para calcular o produto bit a bit, levando em consideração o transporte de bits (carry) entre as operações. O resultado final é obtido combinando os produtos parciais gerados pelas portas AND e ajustando os bits de acordo com as regras da aritmética binária.